In [1]:
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from eli5.sklearn import PermutationImportance
from catboost import CatBoostRegressor,Pool
import matplotlib.patches as patch
import matplotlib.pyplot as plt
from sklearn.svm import NuSVR
from scipy.stats import kurtosis
from scipy.stats import skew
from scipy.stats import norm
from scipy import linalg
from sklearn import tree
from sklearn import svm
import lightgbm as lgb
#import xgboost as xgb
from tqdm import tqdm
import seaborn as sns
import pandas as pd
import numpy as np
import graphviz
import warnings
import random
import eli5
import shap  # package used to calculate Shap values
import time
import glob
import sys
import os
from scipy.signal import hilbert
from scipy.signal import hann
from scipy.signal import convolve
from scipy import stats
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import LinearRegression

/Users/wangrunqiu/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/Users/wangrunqiu/anaconda3/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_9.4.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
warnings.filterwarnings("ignore")

print("Reading csv files...")
train = pd.read_csv('./train.csv' , dtype={'acoustic_data': np.int16, 'time_to_failure': np.float32})
print("Train: rows:{} columns:{}".format(train.shape[0], train.shape[1]))

Reading csv files...
Train: rows:629145480 columns:2


In [5]:
rows = 150_00
segments = int(np.floor(train.shape[0] / rows)) # 41940
print(segments)

X_train = pd.DataFrame(index=range(segments), dtype=np.float64) # 使用对应的特征
y_train = pd.DataFrame(index=range(segments), dtype=np.float64,
                       columns=['time_to_failure'])
submission = pd.read_csv('./sample_submission.csv', index_col='seg_id')

X_train =pd.read_csv('./data/41940samples/X_train_41940samples_138features_filtered.csv',index_col=0)
X_test = pd.read_csv('./data/41940samples/X_test_41940samples_138features_filtered.csv',index_col=0)
y_train= pd.read_csv('./data/41940samples/y_train_41940sasmples.csv',index_col=0)

print(submission.shape)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)

#y_train

scale = 1  # test:使滑动窗口缩小10倍
for segment in range(segments):
    # y_train and X_train
    #     if segment * rows / scale + rows > train.shape[0]:
    #         break
    seg = train.iloc[int(segment * rows / scale): int(segment * rows / scale + rows)]
    x = pd.Series(seg['acoustic_data'].values)
    y = seg['time_to_failure'].values[-1]
    y_train.loc[segment, 'time_to_failure'] = y
#print(X_train.head())
#print(X_test.head())
print(y_train.shape)
#print("submission has: rows:{} cols:{}".format(submission.shape[0], submission.shape[1]))
print("Reading ends.")


41943
(2624, 1)
(41940, 138)
(2624, 138)
(41940, 1)


SystemExit: 

In [6]:
# 正则化
print("FillNA and standardScalar...")
means_dict = {}
for col in X_train.columns:
    if X_train[col].isnull().any():
        print(col)
        mean_value = X_train.loc[X_train[col] != -np.inf, col].mean()
        X_train.loc[X_train[col] == -np.inf, col] = mean_value
        X_train[col] = X_train[col].fillna(mean_value)
        means_dict[col] = mean_value

for col in X_test.columns:
    if X_test[col].isnull().any():
        X_test.loc[X_test[col] == -np.inf, col] = means_dict[col]
        X_test[col] = X_test[col].fillna(means_dict[col])

X = X_train.copy()
y = y_train.copy()
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = pd.DataFrame(scaler.transform(X_train), columns=X_train.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)
print("FillNA and standardScalar ends.")

FillNA and standardScalar...
FillNA and standardScalar ends.


In [7]:
#cross validation
n_fold = 5
folds = KFold(n_splits=n_fold, shuffle=True, random_state=11)
X=X_train_scaled
X_test=X_test_scaled
y=y_train
oof = np.zeros(len(X))
prediction = np.zeros(len(X_test))
scores = []
feature_importance = pd.DataFrame()

In [10]:
print('start')
from sklearn.model_selection import GridSearchCV
param_test1 = {'n_estimators':range(100,1000,10)}
gsearch1 = GridSearchCV(estimator = RandomForestRegressor(), param_grid = param_test1, scoring=None,cv=5)
gsearch1.fit(X,y)
print(gsearch1.best_params_)
print(gsearch1.best_score_)
print('end')

print('start')
param_test4 = {'max_depth':range(1,5), 'min_samples_split':range(2,10),'min_samples_leaf':range(40,60,2)}
gsearch4 = GridSearchCV(estimator = RandomForestRegressor(n_estimators=gsearch1.best_params_['n_estimators']), param_grid = param_test4, scoring=None,cv=5)
gsearch4.fit(X,y)
print(gsearch4.best_params_)
print(gsearch4.best_score_)
print('end')

#refit
print('start')
scores = []
for fold_n, (train_index, valid_index) in enumerate(folds.split(X)):
    # print('Fold', fold_n, 'started at', time.ctime())
    X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
    y_train1, y_valid1 = y.iloc[train_index], y.iloc[valid_index]
    
    model = RandomForestRegressor(n_estimators=gsearch1.best_params_['n_estimators'],bootstrap=True, criterion='mae',
            max_depth=gsearch4.best_params_['max_depth'], max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=gsearch4.best_params_['min_samples_leaf'],
            min_samples_split=gsearch4.best_params_['min_samples_split'], min_weight_fraction_leaf=0.0,
            n_jobs=1, oob_score=True, random_state=10,
            verbose=0, warm_start=False)
    model.fit(X_train, y_train1)
    y_pred_valid = model.predict(X_valid).reshape(-1, )
    score = mean_absolute_error(y_valid1, y_pred_valid)
    y_pred = model.predict(X_test).reshape(-1, )
    scores.append(score)

a = 0
for s in scores:
    a+=s
print(a/5)
print('end')

start
{'n_estimators': 28}
0.3833472280584507
end
start
{'max_depth': 2, 'min_samples_leaf': 50, 'min_samples_split': 9}
0.4523088074060052
end
start
2.089236264226993
end
